## 1. Install the twitter library in python

In [4]:
!pip install tweepy

## 2. Install library for JSON

In [5]:
!pip install simplejson

## 3. Install sentiment analysis library

In [3]:
# Whatever library you use
!pip install textblob

# Streaming tweets and perform some data analysis


### Setting up and running a streaming crawler

In [4]:
import tweepy
import simplejson as json
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from textblob import TextBlob
 
#Complete with your keys 

consumer_key = 'aCqrloIHCuvw8rX8sON8B551T'
consumer_secret = 'CkkLBIClzyC2w00oHOateou4wxJ2IberPKXWR8WpQEi0T5Gh2s'
access_token = '173960222-v34xiH0gEFqXCoVvpnh3JgMp8xwVukAX61rqazBc'
access_secret = '9ey7wPBUkPJdu2ZxaBoOUr43dmFCj2Fql1MxBCPKEoVFg'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
print("Starting code")
print(auth)
 

class MyListener(StreamListener):
    
    def __init__(self, filename, api=None):
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        self.filename = filename

    def on_data(self, data):
        try:
            with open(self.filename, 'a') as f:
                #print("test")
                if json.loads(data).get('place'):
                    print(json.loads(data)['place']['country'])
                    if json.loads(data)['place']['country'].lower() in ['spain', 'españa', 'espana']:
                        f.write(data) # This will store the whole JSON data in the file, you can perform some JSON filters
                        twitter_text = json.loads(data)['text'] # You can also print your tweets here
                        print(twitter_text)
                        self.num_tweets += 1

                # Just to limit the number of tweets collected to check the 
                # program at the beginning, then increase the limit
                if self.num_tweets < 100:
                    return True
                else:
                    print('______________ END ', self.filename)
                    return False
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

 
    def on_error(self, status):
        print('Error :', status.place)
        return False
    
"""twitter_stream = Stream(auth, MyListener('gobierno2.json'))
twitter_stream.filter(track=["gobierno", "elecciones"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('pp.json'))
twitter_stream.filter(track=["pp"], languages=["es"]) # Add your keywords and other filters


twitter_stream = Stream(auth, MyListener('psoe.json'))
twitter_stream.filter(track=["psoe"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('cat.json'))
twitter_stream.filter(track=["cataluña", "catalunya"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('vox.json'))
twitter_stream.filter(track=["vox"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('ciudadanos.json'))
twitter_stream.filter(track=["ciudadanos", "ciutadans"], languages=["es"]) # Add your keywords and other filters
"""
twitter_stream = Stream(auth, MyListener('podemos.json'))
twitter_stream.filter(track=["podemos"], languages=["es"]) # Add your keywords and other filters

print('_______ End _______')



Starting code


'twitter_stream = Stream(auth, MyListener(\'gobierno2.json\'))\ntwitter_stream.filter(track=["gobierno", "elecciones"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_stream = Stream(auth, MyListener(\'pp.json\'))\ntwitter_stream.filter(track=["pp"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_stream = Stream(auth, MyListener(\'psoe.json\'))\ntwitter_stream.filter(track=["psoe"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_stream = Stream(auth, MyListener(\'cat.json\'))\ntwitter_stream.filter(track=["cataluña", "catalunya"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_stream = Stream(auth, MyListener(\'vox.json\'))\ntwitter_stream.filter(track=["vox"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_stream = Stream(auth, MyListener(\'ciudadanos.json\'))\ntwitter_stream.filter(track=["ciudadanos", "ciutadans"], languages=["es"]) # Add your keywords and other filters\n\ntwitter_str

### Store the JSON data in a CSV for analysing

In [84]:
import simplejson as json

# Create the CSV file
#files = ["gobierno", "cat", "podemos", "pp", "vox", "ciudadanos", "psoe"]
files = ["pp"]

for file in files:
    with open (file + ".csv", 'w', encoding ='utf-8') as csv:
        # Write the title of the columns (features) that you want to store in the CSV file
        csv.write('id,'+'created_at,'+'text'+'\n')

        # Copy the data from the JSON file
        with open(file + ".json", 'r', encoding ='utf-8') as jsonfile:
            for tweet in jsonfile: 
                data = json.loads(tweet)

                # The int values should be converted to strings
                csv.write(str(data['id'])+',')
                csv.write(str(data['created_at'])+',') 
                csv.write((str(data['text']).replace('\n', "").replace(',', ""))) 
                #csv.write(str(data['place']['country'])) 
                csv.write('\n')
            

### Load the previous CSV into pandas

In [85]:
import pandas as pd
tweets_all = pd.read_csv('pp.csv', index_col = 0, encoding='utf-8')
tweets_all.head(100)
#count=0
#for tweet in tweets_all:
    


,created_at,text
id,,
1219242646469775360,Mon Jan 20 12:57:54 +0000 2020,El #PinParental no se aplica a asignaturas cur...
1219243127132868608,Mon Jan 20 12:59:48 +0000 2020,@Santi_ABASCAL Por fin un partido que defiende...
1219243480255467520,Mon Jan 20 13:01:12 +0000 2020,La ocurrencia de Vox del veto parental le esta...
1219243557908877314,Mon Jan 20 13:01:31 +0000 2020,@meneses @nostraeuropa @diazvillanueva Diazvil...
1219244207048658944,Mon Jan 20 13:04:06 +0000 2020,Estoy contigo todos unidos derrotaremos al ene...
...,...,...
1219312809156235264,Mon Jan 20 17:36:42 +0000 2020,Vox se rie del PP...o el PP se deja ser un haz...
1219313321305952256,Mon Jan 20 17:38:44 +0000 2020,@pmarzo1 @pdiaz222f @eldiarioe ¿Torear?. Parec...
1219313733471744000,Mon Jan 20 17:40:22 +0000 2020,@SimnMuoz @MasPais_Es @ierrejon Los maleducado...


### Analysing the polarity of the tweets

In [100]:
!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gabi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [93]:
from textblob import TextBlob
import re, tweepy
from nltk.corpus import stopwords



def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        #return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('spanish')) + r')\b\s*')
        return pattern.sub('', tweet)


def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'                               

def featched_tweets(tweets_):
    try:
        tweets_clean = []
        for tweet in tweets_:
            # empty dictionary to store required params of a tweet 
            parsed_tweet = {} 

            # saving text of tweet 
            parsed_tweet['text'] = tweet 
            # saving sentiment of tweet 
            parsed_tweet['sentiment'] = get_tweet_sentiment(tweet)

            # appending parsed tweet to tweets list 
            tweets_clean.append(parsed_tweet) 

        # return parsed tweets 
        return tweets_clean
        
    except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))
                               
def main(): 
    tweets = featched_tweets(tweets_all["text"])
    # picking positive tweets from tweets found before
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    neutral = list(filter(lambda l: l not in ntweets and l not in ptweets, tweets))
    print("Neutral tweets percentage: {} % ".format(100*len(neutral)/len(tweets)))
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 

if __name__ == "__main__": 
    main()                                
        

Positive tweets percentage: 0.0 %
Negative tweets percentage: 1.1363636363636365 %
Neutral tweets percentage: 98.86363636363636 % 


Positive tweets:


Negative tweets:
@beaveil @BertoEsquer Ojo y en base a unos valores que ordena la LOMCE que una ley del PP!


### Generating a wordcloud

In [38]:
!pip install wordcloud

     |████████████████████████████████| 368kB 901kB/s eta 0:00:01


In [101]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

#tweets = map(clean_tweet, tweets_all["text"])
tweets = [clean_tweet(tweet) for tweet in tweets_all['text']]

text = " ".join(review for review in tweets)
print("There are {} words in the combination of all review. ".format(len(text)))

#create stopword list

stopwords = set(STOPWORDS)
stopwords.update(["http","https", "co"])

# generate wordcloud image
wordcloud = WordCloud(stopwords=stopwords,background_color="white").generate(text)

#Display generated image:

plt.figure(figsize=(10,15))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

AttributeError: 'set' object has no attribute 'words'

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

text = " ".join(review for review in tweets.text)
print("There are {} words in the combination of all review. ".format(len(text)))

#create stopword list

stopwords = set(STOPWORDS)
stopwords.update(["http","https", "co"])

# generate wordcloud image
wordcloud = WordCloud(stopwords=stopwords,background_color="white").generate(text)

#Display generated image:

plt.figure(fgisize=(10,15))
plt.imshow(worldcloud, interplotation'bilinear')
plt.axis("off")
plt.show()



### Your own analysis

In [ ]:
...